In [1]:
!pip install neo4j
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import requests
import json
from neo4j import GraphDatabase

In [3]:
url = 'https://stapi.co/api/v1/rest/episode/search'

params = {
    'pageSize': 50 ,
    'pageNumber': 1
}

episodes = pd.DataFrame()
response = requests.get(url, params=params)
totalPages = int(response.json()['page']['totalPages'])

for i in range(1, int(totalPages)):
    params['pageNumber'] = i
    response = requests.get(url, params=params)
    epis = pd.json_normalize(response.json(), 'episodes')
    epis = epis[epis['series.uid'] == 'SEMA0000073238']
    epis = epis ['uid']
    # print(epis)
    episodes = pd.concat([episodes, epis])

In [4]:
url = 'https://stapi.co/api/v1/rest/episode'
char_params = {}
char_ids = {'CHMA0000278202',}

for uid in episodes['uid']:
    char_params['uid'] = uid
    response = requests.get(url, params=char_params)
    characters = pd.json_normalize(response.json())
    characters = characters['episode.characters'][0]
    for character in characters:
        char_ids.add(character['uid'])

In [5]:
characters = pd.DataFrame()
url = 'https://stapi.co/api/v1/rest/character/'
for uid in char_ids:
    char_params['uid'] = uid
    response = requests.get(url, params=char_params)
    characF = pd.json_normalize(response.json())
    characF = characF[['character.uid', 'character.name', 'character.gender', 'character.episodes', 'character.titles', 'character.organizations']]
    characters = pd.concat([characters, characF])
    

In [6]:
characters_relations = pd.DataFrame()
url = 'https://stapi.co/api/v1/rest/character/'
for uid in char_ids:
    char_params['uid'] = uid
    response = requests.get(url, params=char_params)
    characF = pd.json_normalize(response.json())
    characF = characF[['character.uid', 'character.characterRelations']]
    characters_relations = pd.concat([characters_relations, characF])
characters_relations

,character.uid,character.characterRelations
0,CHMA0000008989,[]
0,CHMA0000032894,[]
0,CHMA0000021866,[]
0,CHMA0000040581,[]
0,CHMA0000289509,"[{'type': 'Relative', 'source': {'uid': 'CHMA0..."
...,...,...
0,CHMA0000021760,[]
0,CHMA0000025507,"[{'type': 'Creator', 'source': {'uid': 'CHMA00..."
0,CHMA0000019130,[]
0,CHMA0000024051,[]


In [7]:
URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "password")

In [8]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    with driver.session(database="neo4j") as session:
        for index, row in characters.iterrows():
            charac_query = session.run("CREATE (:Character{uid: $uid, name: $name, gender: $gender})", uid = row['character.uid'], name = row['character.name'], gender = row['character.gender'], database = 'neo4j')
            for episode in row['character.episodes']:
                char_relations = session.run("MATCH (c:Character {uid: $uid}) MATCH(e:Episode {uid: $e_uid}) CREATE (c)-[:Appears_In]->(e)", uid = row['character.uid'], e_uid = episode['uid'])
            for title in row['character.titles']:
                char_titles = session.run("MATCH (c:Character {uid: $uid}) MERGE (t:Title{uid: $t_uid, name: $t_name}) MERGE (c)-[:Title]->(t)", uid = row['character.uid'], t_uid = title['uid'], t_name = title['name'])
            for organization in row['character.organizations']:
                charac_oraganization = session.run("MATCH (c:Character {uid: $uid}) MERGE (o:Organization{uid: $o_uid, name:$o_name}) MERGE (c)-[:Part_Of]->(o)", uid = row['character.uid'], o_uid = organization['uid'], o_name = organization['name'])
                            
                            

In [9]:
characters_relations[characters_relations['character.uid'] == 'CHMA0000128914']
        

,character.uid,character.characterRelations
0,CHMA0000128914,"[{'type': 'Spouse', 'source': {'uid': 'CHMA000..."


In [10]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    with driver.session(database='neo4j') as session:
        for index, row in characters_relations.iterrows():
                for relation in row['character.characterRelations']:
                    relations_query = "MATCH (s:Character{uid: $sourceUid}) MATCH (t:Character{uid: $targetUid}) MERGE (s)-[:"+ relation['type'].replace(' ', '_').replace('-','_') + "]->(t)"
                    print(relations_query)
                    relations_result = session.run(relations_query, sourceUid = relation['source']['uid'], targetUid = relation['target']['uid'])

MATCH (s:Character{uid: $sourceUid}) MATCH (t:Character{uid: $targetUid}) MERGE (s)-[:Relative]->(t)
MATCH (s:Character{uid: $sourceUid}) MATCH (t:Character{uid: $targetUid}) MERGE (s)-[:Father]->(t)
MATCH (s:Character{uid: $sourceUid}) MATCH (t:Character{uid: $targetUid}) MERGE (s)-[:Child]->(t)
MATCH (s:Character{uid: $sourceUid}) MATCH (t:Character{uid: $targetUid}) MERGE (s)-[:Original_character]->(t)
MATCH (s:Character{uid: $sourceUid}) MATCH (t:Character{uid: $targetUid}) MERGE (s)-[:Child]->(t)
MATCH (s:Character{uid: $sourceUid}) MATCH (t:Character{uid: $targetUid}) MERGE (s)-[:Ancestor]->(t)
MATCH (s:Character{uid: $sourceUid}) MATCH (t:Character{uid: $targetUid}) MERGE (s)-[:Relative]->(t)
MATCH (s:Character{uid: $sourceUid}) MATCH (t:Character{uid: $targetUid}) MERGE (s)-[:Father]->(t)
MATCH (s:Character{uid: $sourceUid}) MATCH (t:Character{uid: $targetUid}) MERGE (s)-[:Child]->(t)
MATCH (s:Character{uid: $sourceUid}) MATCH (t:Character{uid: $targetUid}) MERGE (s)-[:Father]-